## Recreating the following for classification problem
https://www.tensorflow.org/text/tutorials/transformer

Need tensorflow-text for using the BERT model tokenizer

In [2]:
# Install the nightly version of TensorFlow to use the improved
# masking support for `tf.keras.layers.MultiHeadAttention`.
!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
!pip install -q -U tensorflow-text tensorflow

     |████████████████████████████████| 578.0 MB 15 kB/s 
     |████████████████████████████████| 1.7 MB 40.6 MB/s 
     |████████████████████████████████| 5.9 MB 51.6 MB/s 
     |████████████████████████████████| 438 kB 67.7 MB/s 


Import the necessary modules:

In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_text
from sklearn.utils import shuffle 

Doing necessary data preprocessing

In [4]:
os.listdir('.')

['.config', 'twitter_validation.csv', 'twitter_training.csv', 'sample_data']

In [5]:
training_data_loc = "./twitter_training.csv"
val_data_loc = "./twitter_validation.csv"

In [6]:
df = pd.read_csv(training_data_loc)
df

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [7]:
df = df[df.columns[2:]]

In [8]:
df = df.rename(columns={"Positive":"Sentiment", "im getting on borderlands and i will murder you all ,":"sentence"})

In [9]:
df['Sentiment'].value_counts()

Negative      22542
Positive      20831
Neutral       18318
Irrelevant    12990
Name: Sentiment, dtype: int64

In [10]:
df = df[df.apply(lambda x: (x["Sentiment"] == "Positive") or (x["Sentiment"] == "Negative"), axis=1)]

In [12]:
df['Sentiment'].value_counts()

Negative    22542
Positive    20831
Name: Sentiment, dtype: int64

In [13]:
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == "Positive" else 0)

In [14]:
df = shuffle(df)

df

,Sentiment,sentence
40793,1,It's Thursday and we be gamin' for the greater...
56600,0,Thinks attackers in RhandlerR with defusers sh...
55352,0,Fuck call of duty after the update
57086,0,@Rainbow6Game so I get a connection failure fr...
65490,1,twitter.com / AP / status / 1315.... could sti...
...,...,...
55445,0,@CallofDuty has gotten over 156GB. . . Absolut...
36777,0,... Is this why my stupid email went extra scr...
40455,1,Battlefield V.. I fucking hate dice and EA War...
50423,0,Just downloaded FIFA 20 and by God this game s...


In [15]:
df.isna().value_counts()

Sentiment  sentence
False      False       43012
           True          361
dtype: int64

In [16]:
df.dropna(inplace=True) # dropping empty sentences

In [17]:
df

,Sentiment,sentence
40793,1,It's Thursday and we be gamin' for the greater...
56600,0,Thinks attackers in RhandlerR with defusers sh...
55352,0,Fuck call of duty after the update
57086,0,@Rainbow6Game so I get a connection failure fr...
65490,1,twitter.com / AP / status / 1315.... could sti...
...,...,...
55445,0,@CallofDuty has gotten over 156GB. . . Absolut...
36777,0,... Is this why my stupid email went extra scr...
40455,1,Battlefield V.. I fucking hate dice and EA War...
50423,0,Just downloaded FIFA 20 and by God this game s...


In [18]:
labels = df['Sentiment']
features = df['sentence']

## Tokenizer
using subword tokenizer implementation from Bert model

In [19]:
model_name = 'ted_hrlr_translate_pt_en_converter'
tf.keras.utils.get_file(
    f'{model_name}.zip',
    f'https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip',
    cache_dir='.', cache_subdir='', extract=True
)

184801/184801 [==============================] - 0s 0us/step


'./ted_hrlr_translate_pt_en_converter.zip'

In [20]:
tokenizers = tf.saved_model.load(model_name)

The `tf.saved_model` contains two text tokenizers, one for English and one for Portuguese. We just need the English one

In [ ]:
[item for item in dir(tokenizers.en) if not item.startswith('_')]

['detokenize',
 'get_reserved_tokens',
 'get_vocab_path',
 'get_vocab_size',
 'lookup',
 'tokenize',
 'tokenizer',
 'vocab']

In [28]:
sample = features.sample(n=5)
sample

72258                               This ACS12 is amazing.
5781     remembered im calling amazon friend to f see w...
63266             No fucking way... is DHop PG 103 rating?
16310                Yes - both love @DOTA2 redd.it/jl7402
59918                   I am moderately interested in this
Name: sentence, dtype: object

In [29]:
text_sample = sample.iloc[0]
print(text_sample)

This ACS12 is amazing.


In [30]:
encoded = tokenizers.en.tokenize(sample)
# print(encoded)
print('Padded and encoded embeddings:')
for row in encoded.to_list():
  print(row)

Padded and encoded embeddings:
[2, 81, 37, 6409, 4824, 80, 452, 15, 3]
[2, 4760, 45, 504, 1926, 2150, 660, 73, 42, 125, 192, 45, 191, 4937, 6067, 45, 145, 677, 91, 440, 92, 71, 2569, 72, 45, 504, 426, 3758, 1209, 54, 269, 93, 48, 2564, 3]
[2, 154, 42, 2713, 138, 152, 15, 15, 15, 80, 5658, 4871, 52, 651, 308, 1223, 5057, 30, 3]
[2, 480, 14, 421, 302, 31, 2115, 303, 1222, 765, 178, 15, 76, 16, 46, 392, 1122, 6381, 1222, 3]
[2, 45, 340, 3826, 1803, 188, 821, 77, 81, 3]


The padded 2 and 3 stands for [START] and [END]

In [31]:
round_trip = tokenizers.en.detokenize(encoded)

print('Original text:')
for line in round_trip.numpy():
  print(line.decode('utf-8'))

Original text:
this acs12 is amazing .
remembered im calling amazon friend to f see why i got charged bc i couldnt do anything on the app and im having anxiety attack rn with lol
no fucking way . . . is dhop pg 103 rating ?
yes - both love @ dota2 redd . it / jl7402
i am moderately interested in this


## Set up a data pipeline with `tf.data`

The following function takes batches of text as input, and converts them to a format suitable for training. 

1. It tokenizes them into ragged batches.
2. It trims each to be no longer than `MAX_TOKENS`.
3. It splits the output (English) tokens into inputs and labels. THese shifted by one step so that the `label` at each location is the id of the next token.
4. It converts the `RaggedTensor`s to padded dense `Tensor`s.
5. It returns an `(inputs, labels)` pair.


# **Problem here!!!**
Cannot prepare batches as required. I don't know how to fix it.

In [34]:
MAX_TOKENS=128
def prepare_batch(en):
    en = tokenizers.en.tokenize(en)
    en = en[:, :(MAX_TOKENS+1)]
    en_inputs = en[:, :-1].to_tensor()
    return en

In [33]:
sample = df.sample()
sample

,Sentiment,sentence
43668,0,@ PUBG _ Support bro. please stop forcing ever...


In [35]:
prepare_batch(sample['sentence'])

<tf.RaggedTensor [[2, 31, 52, 4118, 651, 35, 786, 38, 1548, 15, 1102, 441, 85, 1668, 506,
  73, 528, 49, 2363, 2489, 258, 13, 148, 1560, 106, 844, 7, 77, 844, 7,
  74, 71, 139, 15, 96, 86, 21, 1703, 15, 523, 81, 3529, 121, 172, 188, 15,
  1102, 15, 3]]>

In [ ]:
tru_ds = prepare_batch(df["sentence"], df['Sentiment'])

In [ ]:
ragged_batches = tru_ds[0]
labels = tru_ds[1]

print(ragged_batches)
print(labels)

<tf.RaggedTensor [[2, 82, 31, 5759, 203, 141, 37, 174, 52, 787, 15, 172, 497, 45, 1650, 159,
  99, 425, 80, 375, 4550, 811, 15, 15, 72, 45, 94, 89, 100, 180, 167, 415,
  100, 45, 106, 15, 15, 15, 2361, 308, 80, 37, 3455, 269, 15, 4184, 130, 74,
  565, 222, 43, 5547, 240, 4, 52, 483, 15, 1693, 15, 3456, 16, 40, 601,
  1394, 2204, 787, 1205, 1010, 271, 1061, 258, 3]                           ,
 [2, 81, 278, 80, 5011, 15, 15, 3],
 [2, 31, 41, 1107, 5171, 88, 31, 41, 2489, 5934, 269, 1192, 392, 135, 965,
  80, 37, 4952, 3236, 13, 6599, 4938, 43, 256, 3917, 13, 76, 9, 55, 100,
  107, 1686, 74, 79, 4952, 2159, 5562, 1413, 1175, 150, 1868, 4185, 15, 79,
  883, 94, 9, 56, 1009, 85, 19, 484, 73, 2958, 79, 4952, 6237, 2565, 88,
  122, 74, 457, 15, 91, 173, 726, 72, 547, 50, 2875, 303, 193, 15, 4952,
  3388, 4, 3]                                                              ,
 ...,
 [2, 45, 153, 154, 642, 1370, 299, 45, 91, 100, 3431, 92, 1184, 120, 76,
  196, 98, 195, 10, 3455, 677, 11, 56, 392,

In [ ]:
df.iloc[1]

Sentiment                              1
sentence     This place is magnificent..
Name: 13384, dtype: object

In [ ]:
ragged_batches[1], labels.iloc[1]

(<tf.Tensor: shape=(8,), dtype=int64, numpy=array([   2,   81,  278,   80, 5011,   15,   15,    3])>,
 1)

In [ ]:
tokenizers.en.detokenize([ragged_batches[1]])

<tf.Tensor: shape=(1,), dtype=string, numpy=array([b'this place is magnificent . .'], dtype=object)>

## Copied code 
we need to change it up a bit so we can use same definitions for our own classification problem

### Define the Embedding and Positional encoding

In [36]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)
  
  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

  return tf.cast(pos_encoding, dtype=tf.float32)

In [37]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x


In [38]:
embed_en = PositionalEmbedding(vocab_size=tokenizers.en.get_vocab_size(), d_model=512)
embed_en

In [40]:
# This is where the error pops up. I persume it is because of the way we are preparing batches.

embed_en(prepare_batch(sample))

ValueError: ignored

### Define the feed forward network

Define a function for the point-wise feed-forward network that you'll use later.

The network consists of two linear layers (`tf.keras.layers.Dense`) with a ReLU activation in-between:

In [41]:
def point_wise_feed_forward_network(
  d_model, # Input/output dimensionality.
  dff # Inner-layer dimensionality.
  ):

  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # Shape `(batch_size, seq_len, dff)`.
      tf.keras.layers.Dense(d_model)  # Shape `(batch_size, seq_len, d_model)`.
  ])

### Define the encoder layer

In [42]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               dropout_rate=0.1
               ):
    super().__init__()


    # Multi-head self-attention.
    self.mha = tf.keras.layers.MultiHeadAttention(
        num_heads=num_attention_heads,
        key_dim=d_model, # Size of each attention head for query Q and key K.
        dropout=dropout_rate,
        )
    # Point-wise feed-forward network.
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    # Layer normalization.
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    # Dropout for the point-wise feed-forward network.
    self.dropout1 = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x, training, mask):

    # A boolean mask.
    if mask is not None:
      mask1 = mask[:, :, None]
      mask2 = mask[:, None, :]
      attention_mask = mask1 & mask2
    else:
      attention_mask = None

    # Multi-head self-attention output (`tf.keras.layers.MultiHeadAttention `).
    attn_output = self.mha(
        query=x,  # Query Q tensor.
        value=x,  # Value V tensor.
        key=x,  # Key K tensor.
        attention_mask=attention_mask, # A boolean mask that prevents attention to certain positions.
        training=training, # A boolean indicating whether the layer should behave in training mode.
        )

    # Multi-head self-attention output after layer normalization and a residual/skip connection.
    out1 = self.layernorm1(x + attn_output)  # Shape `(batch_size, input_seq_len, d_model)`

    # Point-wise feed-forward network output.
    ffn_output = self.ffn(out1)  # Shape `(batch_size, input_seq_len, d_model)`
    ffn_output = self.dropout1(ffn_output, training=training)
    # Point-wise feed-forward network output after layer normalization and a residual skip connection.
    out2 = self.layernorm2(out1 + ffn_output)  # Shape `(batch_size, input_seq_len, d_model)`.

    return out2

### Define the encoder

In [43]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self,
               *,
               num_layers,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               input_vocab_size, # Input (Portuguese) vocabulary size.
               dropout_rate=0.1
               ):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    # Embeddings + Positional encoding
    self.pos_embedding = PositionalEmbedding(input_vocab_size, d_model)

    # Encoder layers.
    self.enc_layers = [
        EncoderLayer(
          d_model=d_model,
          num_attention_heads=num_attention_heads,
          dff=dff,
          dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    # Dropout.
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  # Masking.
  def compute_mask(self, x, previous_mask=None):
    return self.pos_embedding.compute_mask(x, previous_mask)

  def call(self, x, training):

    seq_len = tf.shape(x)[1]

    # Sum up embeddings and positional encoding.
    mask = self.compute_mask(x)
    x = self.pos_embedding(x)  # Shape `(batch_size, input_seq_len, d_model)`.
    # Add dropout.
    x = self.dropout(x, training=training)

    # N encoder layers.
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x  # Shape `(batch_size, input_seq_len, d_model)`.